# Characterizing development with growth modeling

### Environment

In [ ]:
%load_ext lab_black
import pandas as pd
import altair as alt
import numpy as np
from scipy.optimize import curve_fit
from IPython.display import clear_output

### Ingest, tidy

In [ ]:
df = pd.read_csv("1250_sims.csv", index_col=0)

# Rename variables to use old codes
# Should avoid variable with ".", since it will block some convienent function in pandas
df.rename(
    columns={
        "ID": "code_name",
        "Trial.Scaled": "epoch",
        "Hidden": "hidden_units",
        "PhoHid": "cleanup_units",
        "Pnoise": "p_noise",
        "Epsilon": "learning_rate",
        "Type": "cond",
        "Measure": "measure",
        "Score": "score",
        "Freq": "cond_freq",
        "Cons": "cond_cons",
    },
    inplace=True,
)

### Grand mean development over time

In [ ]:
gmdf = df.groupby(["epoch", "cond"]).mean().reset_index()
alt.Chart(gmdf).mark_line(point=True).encode(y="score", x="epoch", color="cond")

### Equations

In [ ]:
def linear(x, a, b):
    return a * x + b


def quadratic(x, a, b, c):
    return a * np.power(x, 2) + b * x + c


def cubic(x, a, b, c, d):
    return a * np.power(x, 3) + b * np.power(x, 2) + c * x + d


def vonb(x, max_acc, k, x0):
    """ von Bertalanffy (1938)
    Assume that the rate of growth of an organism declines with size 
    so that the rate of change in length, l,  may be described by:
    dl/dt = K (L_inf - l) or under our context: dy/dx = k (max_acc - y)
    max_acc: Maximum accuracy / upper asymtote
    k: growth rate
    x0: x value where model start to learn
    """
    return max_acc * (1 - np.exp(-k * (x - x0)))


def expg(x, max_acc, k):
    """ Somewhat Exponential growth (no reference, but 
    equavalent to von Bertalanffy with x0 == 0)
    Assume x-intercept at 0
    max_acc: upper asymtote
    k: growth rate
    """
    return max_acc * (1 - np.exp(-k * x))


def janos(x, max_acc, k, i):
    """ Janoschek, A. (1957)
    # Occationally fail to to converge 
    max_acc: Maximum accuracy
    k: growth rate
    i: related to inflection point x location
    """
    return max_acc * (1 - np.exp(-k * np.power(x, i)))

### Fit functions and convienient function

In [ ]:
def get_df(df, code_name, cond, remove_zero=False):
    """
    Convienient function for subsetting data
    """
    data = df.loc[
        (df.code_name == code_name)
        & (df.cond == cond)
        & (df.measure == "Accuracy"),  # Early points are too volatile
        ["epoch", "score"],
    ].reset_index(drop=True)

    if remove_zero:
        data = data.loc[
            data.score > 0,
        ]

    return data


class growth_model:
    def __init__(self, df, growth_fx, bounds, name, robust=False, f_scale=0.02):
        """ Fit growth models
        df: datafile with score (y) and epoch (x)
        growth_fx: growth function to fit
        bounds: model constrain of parameters
        name: model name
        robust: whether to use robust method
        f_scale (only or robust = True): soft residual cutoff for outlier, 
            used in scipy.optimize.least_squares()
        See https://scipy-cookbook.readthedocs.io/items/robust_regression.html
        for more details
        """
        self.df = df
        self.growth_fx = growth_fx
        self.bounds = (-np.inf, np.inf) if bounds == None else bounds
        self.__name__ = name
        self.f_scale = f_scale

        # Create plotting dataset
        self.df["set"] = "actual"
        self.actual = self.df.score
        self.n = len(self.df)

        # Fit model
        self.fit_robust() if robust else self.fit()

        # Predicts
        self.pred = self.growth_fx(self.df.epoch, *self.params)
        self.res = np.sum(np.square(self.pred - self.actual))
        self.adj_res = self.res * 19 / self.n

        model_df = pd.DataFrame(
            {"epoch": self.df.epoch, "score": self.pred, "set": "predict",}
        )
        self.df = pd.concat([self.df, model_df], ignore_index=True)

    def fit(self):

        self.params, _ = curve_fit(
            self.growth_fx, self.df.epoch, self.df.score, bounds=self.bounds
        )

    def fit_robust(self):
        """ Fitting the selected curve with robust method
        """
        self.params, _ = curve_fit(
            self.growth_fx,
            self.df.epoch,
            self.df.score,
            bounds=self.bounds,
            maxfev=10000,
            method="trf",  # Relatively robust method
            loss="soft_l1",  # More robust to outlier
            f_scale=self.f_scale,  # Soft boundary for outlier residual, based on specific data set,
        )

    def plot(self):
        return (
            alt.Chart(self.df)
            .mark_line(point=True)
            .encode(
                y=alt.Y("score", scale=alt.Scale(domain=(0, 1))),
                x=alt.X("epoch", scale=alt.Scale(domain=(0, 1))),
                color="set",
            )
            .properties(
                title=[
                    f"Model: {self.__name__}",
                    f"Parameters: {self.params.round(3)}",
                    f"Residual (SSE): {self.res:.3f}",
                    f"Data point adjusted residual {self.adj_res:.3f}",
                ]
            )
        )


def plot_all_models(df):
    """ Plot all 6 candidate models for case study
    """

    df_no_zero = df.loc[
        df.score > 0,
    ]
    # models
    m1 = growth_model(df, linear, bounds=None, name="linear")
    m2 = growth_model(df, quadratic, None, "Quadratic")
    m3 = growth_model(df, cubic, None, "Cubic")
    m4 = growth_model(df_no_zero, vonb, (0, [1, np.inf, 1]), "EQ1", True, 0.01)
    m5 = growth_model(df_no_zero, expg, (0, [1, np.inf]), "EQ2")
    m6 = growth_model(df, janos, ([0, 0, -np.inf], [1, np.inf, np.inf]), "EQ3")

    # plots
    pg1 = m1.plot() | m2.plot() | m3.plot()
    pg2 = m4.plot() | m5.plot() | m6.plot()

    return pg1 & pg2

### High accuracy sample

In [ ]:
plot_all_models(get_df(df, 65317006, "NW_UN", remove_zero=False))

### Another high accuracy example

In [ ]:
plot_all_models(get_df(df, 65317006, "HF_CON", remove_zero=False))

### Medium

In [ ]:
plot_all_models(get_df(df, 64195100, "HF_INC", remove_zero=False))

### Low accuracy example

In [ ]:
plot_all_models(get_df(df, 64195080, "LF_INC", remove_zero=False))

### Extremely low accuracy example

In [ ]:
plot_all_models(get_df(df, 64195680, "LF_INC", False))

# Fit all models and conditions

In [ ]:
i = 0
n = len(df.code_name.unique())

eq1_results = pd.DataFrame()
eq2_results = pd.DataFrame()
eq3_results = pd.DataFrame()

for m in df.code_name.unique():
    clear_output(wait=True)
    i += 1
    print("Processing model {} of {}".format(i, n))
    for c in df.cond.unique():
        m1 = growth_model(get_df(df, m, c, remove_zero=True), vonb, (0, [1, np.inf, 1]), "EQ1")
        m2 = growth_model(get_df(df, m, c, remove_zero=True), expg, (0, [1, np.inf]), "EQ2")
        m3 = growth_model(get_df(df, m, c), janos, ([0, 0, -np.inf], [1, np.inf, np.inf]), "EQ3")
        
        m1_result = pd.DataFrame(
            {"model": "EQ1", "nadj_residual": m1.adj_res, "max_acc":m1.params[0], 
             "k":m1.params[1], "x0":m1.params[2]}, index=[0])
        
        m2_result = pd.DataFrame(
            {"model": "EQ2", "nadj_residual": m2.adj_res, "max_acc":m2.params[0] , 
             "k":m2.params[1]}, index=[0])
        
        m3_result = pd.DataFrame(
            {"model": "EQ3", "nadj_residual": m3.adj_res, "max_acc":m3.params[0] , 
             "k":m3.params[1], "i":m3.params[2]}, index=[0])
        
        m1_result["code_name"] = m
        m2_result["code_name"] = m
        m3_result["code_name"] = m
        
        m1_result["cond"] = c
        m2_result["cond"] = c
        m3_result["cond"] = c
        
        eq1_results = pd.concat([eq1_results, m1_result])
        eq2_results = pd.concat([eq2_results, m2_result])
        eq3_results = pd.concat([eq3_results, m3_result])

### Merge h-params spec

In [ ]:
model_spec = df.groupby("code_name").mean().reset_index()
model_spec.drop(columns=["epoch", "score"], inplace=True)

eq1_results = eq1_results.reset_index()
eq1_results = eq1_results.merge(model_spec, on="code_name")
eq1_results.to_csv("eq1_results.csv")
print(f"EQ 1 mean residual = {eq1_results.residual.mean():.4f}")

eq2_results = eq2_results.reset_index()
eq2_results = eq2_results.merge(model_spec, on="code_name")
eq2_results.to_csv("eq2_results.csv")
print(f"EQ 2 mean residual = {eq2_results.residual.mean():.4f}")

eq3_results = eq3_results.reset_index()
eq3_results = eq3_results.merge(model_spec, on="code_name")
eq3_results.to_csv("eq3_results.csv")
print(f"EQ 3 mean residual = {eq3_results.residual.mean():.4f}")

### Growth parameter outlier fix
- There is an outlier estimate in model 65317006, NW_UN
- Which caused by an extreme value in the 2nd data point
- maybe it is good to use a more robust method to fit the curve

In [ ]:
i = 0
n = len(df.code_name.unique())

robust_eq1_results = pd.DataFrame()

for m in df.code_name.unique():
    clear_output(wait=True)
    i += 1
    print("Processing model {} of {}".format(i, n))
    for c in df.cond.unique():
        m1 = growth_model(
            get_df(df, m, c, remove_zero=True),
            vonb,
            (0, [1, np.inf, 1]),
            "rEQ1",
            True,
            0.01,
        )
        m1_result = pd.DataFrame(
            {
                "model": "EQ1",
                "adj_residual": m1.adj_res,
                "max_acc": m1.params[0],
                "k": m1.params[1],
                "x0": m1.params[2],
            },
            index=[0],
        )
        m1_result["code_name"] = m
        m1_result["cond"] = c

        robust_eq1_results = pd.concat([robust_eq1_results, m1_result])

In [ ]:
model_spec = df.groupby("code_name").mean().reset_index()
model_spec.drop(columns=["epoch", "score"], inplace=True)

robust_eq1_results = robust_eq1_results.reset_index()
robust_eq1_results = robust_eq1_results.merge(model_spec, on="code_name")
robust_eq1_results.to_csv("req1_results.csv")
print(f"EQ 1 mean residual = {robust_eq1_results.adj_residual.mean():.4f}")

In [ ]:
!sudo poweroff

In [ ]:
plot_all_models(get_df(df, 65317490, "HF_INC", remove_zero=False))